# Useful snippets from other notebooks

## Printing enhancement

In [4]:
import pprint
results = ['spam', 'eggs', 'lumberjack', 'knights', 'ni']
print(results)
pprint.pprint(results, width=20, compact=True)
# See docs at https://docs.python.org/3/library/pprint.html

['spam', 'eggs', 'lumberjack', 'knights', 'ni']
['spam', 'eggs',
 'lumberjack',
 'knights', 'ni']


## Restart kernel after install of packages

In [5]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

## Setting OpenAI API key if not present in environment variable

In [9]:
!pip install -q -U openai

In [12]:
import openai
import os
import getpass

# Configuring the environment variable OPENAI_API_KEY
if "OPENAI_API_KEY" not in os.environ:
    # OR set the key here as a variable
    openai.api_key = getpass.getpass("OpenAI API Key:")
    
assert len(openai.Model.list()["data"]) > 0 #Will generate an error if the key is not valid

OpenAI API Key: ········


## Testing if you have a CUDA device setup with pytorch

In [16]:
import torch

In [17]:
# Alternative from pytorch docs that allows for Mac GPU training as well
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

Using cuda device


In [ ]:
torch.cuda.get_device_name()

## To have some 'useful' raw data for RAG stuff, I extracted my own bookmarks

In [19]:
import json
import os

def extract_chrome_bookmarks():
    # Define the path to the Chrome bookmarks file
    # This is the default location on Windows. You may need to adjust this for other operating systems. Also may need to change the user to match yours
    path_to_bookmarks = "/mnt/c/Users/nissa/AppData/Local/Google/Chrome/User Data/Default/Bookmarks"

    # Ensure the file exists
    if not os.path.exists(path_to_bookmarks):
        print("The bookmarks file was not found at the specified location. Ensure Chrome is installed and you have bookmarks saved.")
        return []

    # Load the JSON data
    with open(path_to_bookmarks, 'r', encoding='utf-8') as file:
        bookmarks_data = json.load(file)

    # Define a recursive function to extract bookmarks
    def extract_from_node(node):
        if 'url' in node:
            return [(node['name'], node['url'])]
        else:
            urls = []
            if 'children' in node:
                for child in node['children']:
                    urls.extend(extract_from_node(child))
            return urls

    # Extract the bookmarks
    bookmarks = extract_from_node(bookmarks_data['roots']['bookmark_bar']) 
    bookmarks += extract_from_node(bookmarks_data['roots']['other'])  # Other bookmarks

    return bookmarks

bookmarks_list = extract_chrome_bookmarks()
# Run below if you want to be sure your bookmarks are imported as you expect
# for title, url in bookmarks_list:
    # print(f"{title}: {url}")

## Importing data from Kaggle

> Using [Jeremy Howard's notebook](https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners/) as a learning sample starting point for my modified settings

In [1]:
!pip install kaggle -q

In [2]:
# for working with paths in Python, Jeremy Howard recommends using `pathlib.Path`
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser() #If you don't have the kaggle.json read about this at https://www.kaggle.com/docs/api

In [3]:
competition_name='us-patent-phrase-to-phrase-matching' #change to match competition name

In [4]:
path = Path(f'data/{competition_name}') 

In [5]:
import subprocess
import os

def download_competition_data(competition_name):
    # Create 'data' folder if it doesn't exist
    if not os.path.exists('data'):
        os.makedirs('data')

    # Run the Kaggle CLI command to download the competition data into 'data' folder
    cmd = f"kaggle competitions download -c {competition_name} -p data"
    subprocess.run(cmd, shell=True)

In [6]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '') # This is to check if this notebook is running on Kaggle

In [7]:
# Note if an error, make sure to visit the competition page and accept the rules so you can download the data
if not iskaggle and not path.exists():
    import zipfile,kaggle
    # Replace 'your-competition-name-here' with the desired competition's name
    download_competition_data(competition_name)
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [8]:
!ls {path}

sample_submission.csv  test.csv  train.csv


## Exploring data with Pandas

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv(path/'train.csv')

In [11]:
df.head(5)

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [13]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


## Use only my selected GPU (Prevents out of memory crashes when fine tuning)

In [1]:
!nvidia-smi

Mon Sep 11 09:18:45 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98.01              Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti     On  | 00000000:01:00.0  On |                  N/A |
| N/A   62C    P8               7W /  80W |    723MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
import torch

print("Is cuda available?", torch.cuda.is_available())

print("Is cuDNN version:", torch.backends.cudnn.version())

print("cuDNN enabled? ", torch.backends.cudnn.enabled)

print("Device count?", torch.cuda.device_count())

print("Current device?", torch.cuda.current_device())

print("Device name? ", torch.cuda.get_device_name(torch.cuda.current_device()))

Is cuda available? True
Is cuDNN version: 8700
cuDNN enabled?  True
Device count? 1
Current device? 0
Device name?  NVIDIA TITAN RTX
